In [52]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [53]:
import pandas as pd
from xbbg import blp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from datetime import timedelta as td
import statsmodels.api as sm
import re
import matplotlib.image as image
from ipywidgets import interact, IntSlider, Checkbox, Dropdown, Output, HBox, VBox, interactive, interactive_output, ToggleButton,Text, Button, DatePicker, IntText, ToggleButtons, RadioButtons,SelectMultiple, FloatText
from IPython.display import display, clear_output
import itertools
from scipy import stats
from scipy.optimize import minimize 
from scipy.special import ndtr
import warnings


spot_library = {'eur_6':['EUSA', ' Curncy'],'eur': ['EESWE', ' Curncy'], 'usd': ['USOSFR', ' Curncy'], 
                'gbp': ['BPSWS', ' Curncy'],'chf': ['SFSNT', ' Curncy'],'sek': ['SKSW', ' Curncy'],
                 'nok': ['NKSW', ' Curncy'], 'hkd': ['HDSW', ' Curncy'],'czk': ['CKSW', ' Curncy'],
                 'pln': ['PZSW', ' Curncy'],'ils':['ISSW', ' Curncy'],  'cad':['CDSW', ' Curncy'], 
                 'jpy':['JYSO', ' Curncy'], 'aud': ['ADSW', ' Curncy'],'sgd':['SDSW', ' Curncy'],
                'krw': ['KWSWNI', ' Curncy'],
                'zar': ['SASW', ' Curncy'],
                'nzd': ['NDSW', ' Curncy'],
                'mxn': ['MPSW', ' Curncy']} 

forward_library = {'eur_6': ['EUSA', ' Curncy'], 
                 'eur': ['S0514FS ', ' BLC Curncy'], 
                 'usd': ['S0490FS ', ' BLC Curncy'], 
                 'gbp': ['S0141FS ', ' BLC Curncy'],
                 'chf': ['S0234FS ', ' BLC Curncy'],
                 'sek': ['SD0020FS ', ' BLC Curncy'],
                 'nok': ['SD0313FS ', ' BLC Curncy'],
                 'hkd': ['HDFS', ' Curncy'],
                 'czk': ['S0320FS ', ' BLC Curncy'],
                 'pln': ['S0323FS ', ' BLC Curncy'],
                 'ils': ['ISFS', ' Curncy'],
                 'cad': ['CDFS', ' Curncy'],
                 'jpy': ['S0195FS ', ' BLC Curncy'],
                 'aud': ['SD0302FS ', ' BLC Curncy'],
                'sgd': ['SDFS', ' Curncy'],
                'krw': ['S0205FS ', ' BLC Curncy'],
                'zar': ['SAFS', ' Curncy'],#
                'nzd': ['SD0015FS ', ' BLC Curncy'],
                'mxn': ['SD0083FS ', ' BLC Curncy']} 

################################################# SWAP STRUCTURE BUILDERS ################################################

def cut(some):
    x = 0
    y = some[0]
    while y.isdigit():
        y=some[x]
        x+=1
    return some[:x],some[x:]


def f(tenor):
    x = re.findall(r'\d+',tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 >0:
            num+='1'
        num+=chr(64+(int(x)%12))
    else:
        if len(x) == 1:
            num+='0'
        num +=x
    return num

def t(tenor):
    x = re.findall(r'\d+',tenor)[0]
    num = ''
    if 'm' in tenor.lower():
        if int(x) // 12 >0:
            num+='1'
        else:
            num+='0'
        num+=chr(64+(int(x)%12))
    else:
        if len(x) == 1:
            num+='0'
        num +=x
    return num


def spot_ticker(dex,tenor):
    if dex.lower() == 'mxn':
        y = int(tenor[:-1]) * 13
        num = f'{y//12}{chr(64+(y%12))}'
    else:
        num = tenor[:-1]
    return f'{spot_library[dex][0]}{num}{spot_library[dex][1]}'

def forward_ticker(dex,fwd):
    dex = dex.lower()
    fwd = fwd.lower()
    old = ['eur_6','hkd','ils','cad','sgd','zar']
    
    if cut(fwd)[0] == '0y':
        return spot_ticker(dex,cut(fwd)[1])
    elif dex == 'eur_6':
        F,T = f(cut(fwd)[0]),t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    elif dex in old:
        F,T = t(cut(fwd)[0]),t(cut(fwd)[1])
        return f'{forward_library[dex][0]}{F}{T}{forward_library[dex][1]}'
    else:
        
        return f'{forward_library[dex][0]}{fwd.upper()}{forward_library[dex][1]}'


def swap_structure(dex,structure,start, end = 'today',bps = True):
    f_fly = structure.count('/') == 2
    f_crv = structure.count('/') == 1  
    fly = structure.count('s') == 3
    crv = structure.count('s') == 2
    out = max(2 -sum([i.isalpha() for i in structure]),0)
    
    if f_fly or f_crv:
        legs = [forward_ticker(dex,i) for i in structure.split('/')]
    elif fly or crv:
        legs = [spot_ticker(dex,i) for i in [i+'Y' for i in structure.split('s') if i.isdigit()]]
    else:
        legs = forward_ticker(dex,('0Y'*out) + structure)
        
    df = blp.bdh(legs, flds='px_last', start_date=start,end_date=end).fillna(method ='bfill')  *(100 if bps else 1)
    s  = pd.DataFrame({})
    if f_fly or fly:
        x = (2 * df.iloc[:,1]) - (df.iloc[:,0] + df.iloc[:,2])
    elif f_crv or crv:
        x = df.iloc[:,1] - df.iloc[:,0]
    else:
        x = df.iloc[:,0]
        
    s[f'{dex.upper()} {structure}'] = x
    return s

################################################# BOND STRUCTURE BUILDERS ################################################
def bond_ticker(bond_name, bond_tenor):
    cmb = {'ust':1,'dbr':2,'spgb':8,'frtr':4,'btp':5,'ukt':6}
    return f'RV000{cmb[bond_name]}P {bond_tenor.upper()} BLC Curncy'

def bond_structure(bonds,structure, start, end = 'today'):
    if '-' in bonds:
        s = pd.DataFrame({})
        bond1,bond2 = bonds.split('-')
        s[f'{bonds.upper()} {structure}'] = bond_structure(bond1,structure, start, end)[f'{bond1.upper()} {structure}'] - bond_structure(bond2,structure, start, end)[f'{bond2.upper()} {structure}']
        return s
    else:
        if 'y' in structure.lower(): #bond o/r
            tickers = bond_ticker(bonds.lower(), structure)
            base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
            base.columns = [f'{bonds.upper()} {structure}']
            return base
        
        elif structure.lower().count('s') == 2: #bond curve
            tickers = [bond_ticker(bonds.lower(), f'{i}y') for i in structure.lower().split('s') if i != '']
            base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
            base.columns = tickers
            s = pd.DataFrame({})
            s[f'{bonds.upper()} {structure}'] = base[tickers[1]] - base[tickers[0]]
            return s
        
        elif structure.lower().count('s') == 3: # bond fly
            tickers = [bond_ticker(bonds.lower(), f'{i}y') for i in structure.lower().split('s') if i != '']
            base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
            base.columns = tickers
            s = pd.DataFrame({})
            s[f'{bonds.upper()} {structure}'] = (2*base[tickers[1]]) - (base[tickers[0]] + base[tickers[2]])
            return s

################################################# FUTURES STRUCTURE BUILDERS ################################################

def fut_ticker(dex, contract, name = False):
    x  = ''
    if dex.lower()[:3] == 'eur':
        x = 'ER'
    elif dex.lower() == 'usd':
        x = 'SFR'
    elif dex.lower() == 'gbp':
        x = 'SFI'
    
    if name:
        return x
    else:
        return f'{x}{contract} Comdty'   
           
    
def fut_structure(dex,structure, start, end = 'today'):
    name = fut_ticker(dex.lower(), 0,True)
    
    if structure.lower().count('s') == 2: #fut curve
        tickers = [fut_ticker(dex.lower(), int(i)) for i in structure.lower().split('s') if i != '']
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{name}{structure.replace("s","")}'] = base[tickers[0]] - base[tickers[1]]
        return s
        
    elif structure.lower().count('s') == 3: # fut fly
        tickers = [fut_ticker(dex.lower(), int(i)) for i in structure.lower().split('s') if i != '']
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{name}{structure.replace("s","")}'] = (2*base[tickers[1]]) - (base[tickers[0]] + base[tickers[2]])
        return s
    else:
        tickers = fut_ticker(dex.lower(), structure)
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = [f'{name}{structure}']
        return base

################################################# ASW STRUCTURE BUILDERS ################################################

def get_asw(bond_name,tenor,start, end ='today',euro = 'estr'):
    asso_swap = {'ust':'usd','dbr':'eur','spgb':'eur','frtr':'eur','btp':'eur','ukt':'gbp'}
    
    if asso_swap[bond_name] == 'eur' and euro != 'estr':
        tickers = [bond_ticker(bond_name,tenor), spot_ticker(asso_swap[bond_name]+'_6',tenor)]
    else:
        tickers = [bond_ticker(bond_name,tenor), spot_ticker(asso_swap[bond_name],tenor)]
    base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
    base.columns = tickers
    s = pd.DataFrame({})
    s[f'{tenor.upper()} {bond_name.upper()} ASW'] = base[tickers[1]] - base[tickers[0]]
    return s

def asw_structure(bonds,structure, start, end = 'today',euro = 'estr'):
    if '-' in bonds:
        s = pd.DataFrame({})
        bond1,bond2 = bonds.split('-')
        s[f'{bonds.upper()} ASW {structure}'] = asw_structure(bond1,structure, start, end,euro)[f'{bond1.upper()} ASW {structure}'] - asw_structure(bond2,structure, start, end,euro)[f'{bond2.upper()} ASW {structure}']
        return s
    else:
        if 'y' in structure.lower(): #asw o/r
            return get_asw(bonds,structure,start, end,euro)
        
        elif structure.lower().count('s') == 2: #asw curve
            legs = [ f'{i}y' for i in structure.lower().split('s') if i != '']
            short_leg, long_leg = [get_asw(bonds,i,start,end,euro) for i in legs]
            df = short_leg.join(long_leg, how = 'inner')
            s = pd.DataFrame({})
            s[f'{bonds.upper()} ASW {structure}'] = df.iloc[:,1]  - df.iloc[:,0]  
            return s
        
        elif structure.lower().count('s') == 3: # asw fly
            legs = [ f'{i}y' for i in structure.lower().split('s') if i != '']
            short_leg, belly ,long_leg = [get_asw(bonds,i,start,end,euro) for i in legs]
            short_leg = short_leg.join(belly, how = 'inner')
            df = short_leg.join(long_leg, how = 'inner')
            s = pd.DataFrame({})
            s[f'{bonds.upper()} ASW {structure}'] = (2 * df.iloc[:,1])  - (df.iloc[:,0] + df.iloc[:,2])  
            return s
        

######################################### TICKER HANDLERS #################################################################

def get_other(ticker,attribute,title,start,end='today' ):
    if 'price' in attribute.lower():
        attribute = 'px_last'
    else:
        attribute = 'YLD_YTM_MID'
    base = blp.bdh(ticker,attribute,start,end)
    base.columns = [title]
    return base

######################################### INVOICE SPREAD STRUCTURE BUILDERS ################################################


def get_ivsp(country,start, end ='today',which = 'TUA',euro ='estr'):
    tkr = {'USD':['TUAISPS','3YAISPS','FVAISPS','TYAISPS','UXYAISPS','USAISPS','WNAISPS'],
           'GER':['DUAISP','OEAISP','RXAISP','UBAISP'],
           'ITA':['BTAISP','IKAISP'],
           'FRA':['OATAISP'],
           'GBP':['GAISPO']}
    
    if country.upper() == 'USD' or country.upper() == 'GER' or country.upper() == 'ITA' :
        for i in tkr[country.upper()]:
            if which in i:
                future = i
    else:
        future = tkr[country.upper()][0]
    
    if country.lower() in ['ger','ita','fra'] and euro == 'estr':
        future += 'E'
    final_ticker = f'{future} Comdty'
    base = blp.bdh(final_ticker,'px_last',start,end)
    base.columns = [f'{future} Comdty']    
    return base
                
######################################## MEETING DATES HANDLER ##################################################################
    
def md_swap_ticker(bank, meeting,m_of_m =False):
    central_bank = {'FOMC':['USSOFED',' Curncy'],
                'ECB':['EESF','A Curncy'],
                'MPC':['GPSF','A Curncy']}
    ticker = f'{central_bank[bank.upper()][0]}{meeting}{central_bank[bank.upper()][1]}' 
    if m_of_m:
        return blp.bdp(ticker, 'SW_EFF_DT').values.flatten()[0].strftime('%b')
    else:
        return ticker
    
def get_md_swap(bank,structure,start,end  = 'today'):
    if structure.lower().count('s') == 2:
        tickers = [md_swap_ticker(bank.upper(),int(i)) for i in structure.lower().split('s') if i != ''] 
        lbls = ''.join([md_swap_ticker(bank.upper(),int(i), True) for i in structure.lower().split('s') if i != ''])
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{lbls} {bank.upper()}'] = base[tickers[1]] - base[tickers[0]]
        return s
        
    if structure.lower().count('s') == 3:
        tickers = [md_swap_ticker(bank.upper(),int(i)) for i in structure.lower().split('s') if i != '']
        lbls = ''.join([md_swap_ticker(bank.upper(),int(i), True) for i in structure.lower().split('s') if i != ''])            
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = tickers
        s = pd.DataFrame({})
        s[f'{lbls} {bank.upper()}'] = (2*base[tickers[1]] )- (base[tickers[0]] + base[tickers[2]])
        return s
    else:
        tickers = md_swap_ticker(bank.upper(), structure)
        base = blp.bdh(tickers, 'px_last', start, end).fillna(method ='bfill') * 100
        base.columns = [f'{md_swap_ticker(bank.upper(), structure,True)}{bank.upper()}']
        return base

    
################################################# FX HANDLER ############################################################
def get_fx_pair(pair,start,end='today'):
    ticker = f'{pair.upper()} Curncy'
    base = blp.bdh(ticker,'px_last',start,end)
    base.columns = [pair.upper()]
    return base

######################################## MASTER HANDLER ##################################################################


def function_for_bb(trades,_from,_to):
    master_c = pd.DataFrame({})
    for i in trades.keys():
        if len(trades[i]) != 0:
            for j in trades[i]:
                if i == 'SWAP':
                    h = swap_structure(dex = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)                                
                elif i == 'FUTURE':
                    h = fut_structure(dex = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)                              
                elif i == 'BOND':
                    h = bond_structure(bonds = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)
                elif i ==  'ASW':
                    h = asw_structure(bonds = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to, euro = j[2].lower())
                elif i == 'XMKT SWAP':
                    h1 = swap_structure(dex = j[0].lower() ,structure=j[1].lower() ,start = _from , end = _to)
                    h2 = swap_structure(dex = j[2].lower() ,structure=j[1].lower() ,start = _from , end = _to)
                    h_int = h1.join(h2, how = 'inner')
                    h = pd.DataFrame(h_int.iloc[:,0] - h_int.iloc[:,1], columns = [f'{j[0]}-{j[2]} {j[1]}'])
                elif i == 'MD SWAP':
                    h = get_md_swap(bank = j[0].upper(),structure=j[1].lower() ,start = _from , end = _to)
                elif i == 'FX':
                    h = get_fx_pair(pair = j[0].upper(),start = _from , end = _to)
                elif i == 'OTHER/CIX':
                    h = get_other(ticker = j[0], attribute= j[1], title = j[2], start = _from , end = _to)            

                if master_c.shape == (0,0):
                    master_c = h
                else:
                    master_c = master_c.join(h, how = 'inner')
    return master_c
                            
                            
    

In [54]:

clrs = ['whites','reds','greens','blues']
def fut_presets(dex,clr):
    color = {key : range(1+(clrs.index(key)*4),5+(clrs.index(key)*4)) for key in clrs}
    fin_clr = []
    for i in clr.lower().split(', '):
        fin_clr += list(color[i])
    for i in fin_clr:
        yield (dex.upper(), str(i))

clr_dd = [', '.join([m.capitalize() for m in j]) for i in range(1,len(clrs)+1) for j in itertools.combinations(clrs, i) ]


sp_or_fw = ['Spot Tenors', 'Forward Gaps']
yc = ['(up to 10y)','(up to 15y)','(up to 30y)','(up to 50y)']
srt = ['Spot Curves', 'Spot Flys', 'Forward Curves']
swap_ops = [f'{i} {j}' for i in sp_or_fw for j in yc] + srt
bond_ops = ['Outright', 'Curves', 'Flys']

def swap_presets(dex, build):
    
    spot_front = [f'{i}y' for i in range (1,11)]
    spot_mid = spot_front + ['12y','15y']
    spot_back = spot_mid + ['20y','25y','30y']
    spot_ultraback =spot_back + ['40y','50y']
    
    fwds_front = [f'{i}y1y' for i in range (1,10)]
    fwds_mid = fwds_front +['10y2y','12y3y']
    fwds_back = fwds_mid + ['15y5y','20y5y','25y5y']
    fwds_ultraback = fwds_back + ['30y10y','40y10y']
    
    spot_curves = ['1s2s','2s5s','2s10s','5s10s','5s30s','10s30s']
    spot_flys = ['1s2s3s','1s3s5s', '2s5s10s', '5s10s30s']
    fwds_curves = ['1y1y/5y5y','1y2y/1y10y']
    answer = []
    if 'spot' in build.lower():
        if '10y' in build.lower():
            answer = spot_front
        elif '15y' in build.lower():
            answer = spot_mid
        elif '30y' in build.lower():
            answer = spot_back
        elif '50y' in build.lower():
            answer = spot_ultraback
        elif 'curves' in build.lower():
            answer = spot_curves
        elif 'flys' in build.lower():
            answer = spot_flys 
        else:
            answer = spot_curves + spot_flys
    else:
        if '10y' in build.lower():
            answer = fwds_front
        elif '15y' in build.lower():
            answer = fwds_mid
        elif '30y' in build.lower():
            answer = fwds_back
        elif '50y' in build.lower():
            answer = fwds_ultraback
        else:
            answer = fwds_curves
    for i in answer:
        yield(dex.upper(),i)
        
def bond_spline_presets(bond,build):
    answer = {'Outright': ['2y','5y','10y','30y'],
             'Curves': ['2s5s','2s10s','5s10s','5s30s','10s30s'], 
              'Flys':['2s5s10s', '5s10s30s']}
    for i in answer[build]:
        yield (bond.upper(), i)
        
def asw_spline_presets(bond,build,euro = 'ESTR'):
    answer = {'Outright': ['2y','5y','10y','30y'],
             'Curves': ['2s5s','2s10s','5s10s','5s30s','10s30s'], 
              'Flys':['2s5s10s', '5s10s30s']}
    for i in answer[build]:
        yield (bond.upper(), i,euro)

def custom_presets(broker):
    if broker == "Rob's Weekly":
        ccy = ['GBP', 'EUR', 'USD', 'EUR_6']
        _st = ['1y1y','1y1y/2y1y','5y5y','5y2y/7y3y','5y5y/10y10y','2s5s','2s10s','5s10s','5s30s','10s15s','10s30s']
        return {'SWAP': [(i,j) for i in ccy for j in _st]}
    elif broker == "MR Presets":
        return {'SWAP': [('USD','2y1y/3y2y/5y2y'),('GBP','2y1y/3y2y/5y2y'),('EUR','2y1y/3y2y/5y2y'),('EUR_6','2y1y/3y2y/5y2y'),
                        ]}
        

In [55]:

clrs = ['whites','reds','greens','blues']
def fut_presets(dex,clr):
    color = {key : range(1+(clrs.index(key)*4),5+(clrs.index(key)*4)) for key in clrs}
    fin_clr = []
    for i in clr.lower().split(', '):
        fin_clr += list(color[i])
    for i in fin_clr:
        yield (dex.upper(), str(i))

clr_dd = [', '.join([m.capitalize() for m in j]) for i in range(1,len(clrs)+1) for j in itertools.combinations(clrs, i) ]


sp_or_fw = ['Spot Tenors', 'Forward Gaps']
yc = ['(up to 10y)','(up to 15y)','(up to 30y)','(up to 50y)']
srt = ['Spot Curves', 'Spot Flys', 'Forward Curves']
swap_ops = [f'{i} {j}' for i in sp_or_fw for j in yc] + srt
bond_ops = ['Outright', 'Curves', 'Flys']

def swap_presets(dex, build):
    
    spot_front = [f'{i}y' for i in range (1,11)]
    spot_mid = spot_front + ['12y','15y']
    spot_back = spot_mid + ['20y','25y','30y']
    spot_ultraback =spot_back + ['40y','50y']
    
    fwds_front = [f'{i}y1y' for i in range (1,10)]
    fwds_mid = fwds_front +['10y2y','12y3y']
    fwds_back = fwds_mid + ['15y5y','20y5y','25y5y']
    fwds_ultraback = fwds_back + ['30y10y','40y10y']
    
    spot_curves = ['1s2s','2s5s','2s10s','5s10s','5s30s','10s30s']
    spot_flys = ['1s2s3s','1s3s5s', '2s5s10s', '5s10s30s']
    fwds_curves = ['1y1y/5y5y','1y2y/1y10y']
    answer = []
    if 'spot' in build.lower():
        if '10y' in build.lower():
            answer = spot_front
        elif '15y' in build.lower():
            answer = spot_mid
        elif '30y' in build.lower():
            answer = spot_back
        elif '50y' in build.lower():
            answer = spot_ultraback
        elif 'curves' in build.lower():
            answer = spot_curves
        elif 'flys' in build.lower():
            answer = spot_flys 
        else:
            answer = spot_curves + spot_flys
    else:
        if '10y' in build.lower():
            answer = fwds_front
        elif '15y' in build.lower():
            answer = fwds_mid
        elif '30y' in build.lower():
            answer = fwds_back
        elif '50y' in build.lower():
            answer = fwds_ultraback
        else:
            answer = fwds_curves
    for i in answer:
        yield(dex.upper(),i)
        
def bond_spline_presets(bond,build):
    answer = {'Outright': ['2y','5y','10y','30y'],
             'Curves': ['2s5s','2s10s','5s10s','5s30s','10s30s'], 
              'Flys':['2s5s10s', '5s10s30s']}
    for i in answer[build]:
        yield (bond.upper(), i)
        
def asw_spline_presets(bond,build,euro = 'ESTR'):
    answer = {'Outright': ['2y','5y','10y','30y'],
             'Curves': ['2s5s','2s10s','5s10s','5s30s','10s30s'], 
              'Flys':['2s5s10s', '5s10s30s']}
    for i in answer[build]:
        yield (bond.upper(), i,euro)

def custom_presets(broker):
    if broker == "Rob's Weekly":
        ccy = ['GBP', 'EUR', 'USD', 'EUR_6']
        _st = ['1y1y','1y1y/2y1y','5y5y','5y2y/7y3y','5y5y/10y10y','2s5s','2s10s','5s10s','5s30s','10s15s','10s30s']
        return {'SWAP': [(i,j) for i in ccy for j in _st]}
    elif broker == "MR Presets":
        return {'SWAP': [('USD','2y1y/3y2y/5y2y'),('GBP','2y1y/3y2y/5y2y'),('EUR','2y1y/3y2y/5y2y'),('EUR_6','2y1y/3y2y/5y2y'),
                        ]}
        

In [56]:
def current_swap_level(dex,structure,bps = True):
    f_fly = structure.count('/') == 2
    f_crv = structure.count('/') == 1  
    fly = structure.count('s') == 3
    crv = structure.count('s') == 2
    out = max(2 -sum([i.isalpha() for i in structure]),0)
    
    if f_fly or f_crv:
        legs = [forward_ticker(dex,i) for i in structure.split('/')]
    elif fly or crv:
        legs = [spot_ticker(dex,i) for i in [i+'Y' for i in structure.split('s') if i.isdigit()]]
    else:
        legs = forward_ticker(dex,('0Y'*out) + structure)
        
    df = blp.bdp(legs, flds='px_last').T  *(100 if bps else 1)
    s  = pd.DataFrame({})
    if f_fly or fly:
        x = (2 * df[legs[1]]) - (df[legs[0]] + df[legs[2]])
    elif f_crv or crv:
        x = df[legs[1]] - df[legs[0]]
    else:
        x = df[legs[0]]
        
    s[f'{dex.upper()} {structure}'] = x
    return s



In [57]:
book = pd.read_excel("C:\\blp\\data\\0. systematic ledger.xlsx",index_col = 0)

closable_trades = [book.at[i,'TRADE'] for i in book.index if book.at[i,'EXIT ON'] == 'LIVE' ] 

ECY_W = Dropdown(options = [i.upper() for i in spot_library.keys()], description = 'Currency')
STR_W  = Text(description = 'Structure')
ENTRYD_W = DatePicker(value = dt.today().date(),description ='Entry') 
EXITD_W = DatePicker(value = dt.today().date(),description ='Exit')
TP_W = FloatText(description = 'Target')
SL_W = FloatText(description = 'Stop')
DIR_W = RadioButtons(options = ['PAY', 'REC'], description = 'Direction')
CLOSETRADE_W = Dropdown(options = ['-'] + closable_trades,description = 'Close' )
CLOSE_W = RadioButtons(options = ['At Market','Take Profit', 'Stop out'], description = 'Close trade')

ENTRYL_W = FloatText(description = 'Level')

display(HBox([VBox([ENTRYD_W,ECY_W,STR_W,DIR_W]),
              VBox([ENTRYL_W,TP_W,SL_W]),
              VBox([EXITD_W,CLOSETRADE_W, CLOSE_W])]))




output1 = Output()


LOAD_B = Button(description ='Update Trade Book') 
ADD_B = Button(description ='Add New Trade')
CLOSE_B = Button(description ='Close Trade')
SAVE_B = Button(description ='Save Trade Book')
STATS_B = Button(description ='Show PF Stats')

display(VBox([HBox([LOAD_B,ADD_B,CLOSE_B,SAVE_B]), STATS_B]), output1)


def load_click(b):
    global book
    with output1:
        clear_output()
        for i in book.index:
            if book.at[i,'EXIT ON'] == 'LIVE':
                ecy, _str = book.at[i,'TRADE'].split()
                paid = book.at[i,'DIRECTION'] == 'PAY'
                book.at[i,'CURRENT/LAST LEVEL'] = current_swap_level(ecy, _str).values.flatten()[0]
                chg = round(book.at[i,'CURRENT/LAST LEVEL'] - book.at[i,'ENTRY LEVEL'],2 )
                book.at[i,'P/L (bps)'] = chg if paid else -chg
                    
    
        display(book)

        
def add_click(b):
    global book
    with output1:
        clear_output()
        new_col = book.shape[0] + 1
        
        book.loc[new_col] = [ENTRYD_W.value.strftime('%d %b %y'), f"{ECY_W.value} {STR_W.value}",ENTRYL_W.value,DIR_W.value,ENTRYL_W.value ,TP_W.value, SL_W.value,0., 'LIVE']
        display(book)
        
def close_click(b):
    global book
    with output1:
        clear_output()
        try:
            x = book[(book['TRADE'] ==  CLOSETRADE_W.value)].index.values[-1]
            book.at[x,'EXIT ON'] = EXITD_W.value.strftime('%d %b %y')
            if CLOSE_W.value == 'Take Profit':
                book.at[x,'CURRENT/LAST LEVEL'] = book.at[x,'TP']
                book.at[x,'P/L (bps)'] = abs(book.at[x,'TP'] - book.at[x,'ENTRY LEVEL'])
            
            elif CLOSE_W.value == 'At Market':
                make = book.at[x,'CURRENT/LAST LEVEL'] - book.at[x,'ENTRY LEVEL']
                book.at[x,'P/L (bps)'] = make if book.at[i,'DIRECTION'] == 'PAY' else -make
                
            else:
                book.at[x,'CURRENT/LAST LEVEL'] = book.at[x,'SL']
                book.at[x,'P/L (bps)'] = -abs(book.at[x,'SL'] - book.at[x,'ENTRY LEVEL'])
        except:
            pass
        
        display(book)

def save_click(b):
    global book
    with output1:
        clear_output()
        book.to_excel("C:\\blp\\data\\0. systematic ledger.xlsx")
        try:
            book.to_excel("Y:\\Analytics and Tools\\Swaps\\Rob\\mr_ledger.xlsx")
        except:
            pass
        print('Book saved.')

def stats_click(b):
    with output1:
        clear_output()
        stats = pd.DataFrame({}, index = ['PORTFOLIO'])
        stats.at['PORTFOLIO','Trades'] = f"{int(book.shape[0])}"
        stats.at['PORTFOLIO','Live'] = f"{sum([1 for i in book['EXIT ON'] if i == 'LIVE'])}"
        stats.at['PORTFOLIO','Hit Rate'] =  f"{ round(sum([100 for i in book['P/L (bps)'] if i > 0])/ book.shape[0], 1)} %"
        stats.at['PORTFOLIO','P/L (bps)'] = f"{round(book['P/L (bps)'].sum(),2)} bps"
        stats.at['PORTFOLIO','Average P/L (bps)'] = f"{round(book['P/L (bps)'].mean(),2)} bps"
        stats.at['PORTFOLIO','P/L Vol (bps)'] = f"{round(book['P/L (bps)'].std(),2)} bps"
        stats.at['PORTFOLIO','Sharpe'] = f"{round(book['P/L (bps)'].mean()/book['P/L (bps)'].std(), 2)}"
        stats.T.to_excel('C:\\blp\\data\\stats.xlsx')
        display(stats.T)
        
LOAD_B.on_click(load_click)  
ADD_B.on_click(add_click)
CLOSE_B.on_click(close_click)
SAVE_B.on_click(save_click)
STATS_B.on_click(stats_click)        

Output()

In [58]:
test = book.copy()['P/L (bps)']

In [59]:
test[9] = 11